# PaperEdge 분석
- PaperEdge모델의 네트워크 아키텍처 분석
- 들어온 이미지의 어떤 특징을 추출하고,
어떤것을 지도학습을 통해 학습하고 예측하는가


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
import random
import numpy as np
import cv2

# 자동미분중 이상감지 => 학습시 비활성 할것
torch.autograd.set_detect_anomaly(True)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
# 헬퍼 함수(자주쓰는 컨볼루션 코드)
'''
in_planes: 입력채널수
out_planes: 출력채널수
stride:  컨볼루션이 이동하는 간격
groups: 채널안에서 그룹을 나누어 컨볼루션할때 그룹수
dilation: 커널 필터내에서의 간격
padding: 이미지 경계까지 특징추출할수있게 밖에 패딩추가
'''
# 이미지내 특징 추출용
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

#차원 축소또는 확장용
def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


# 잔차학습(residual) 신경망에 사용되는 요소
'''
resnet의 아키텍처이다
base_width: 네트워크의 기본 너비
downsample: 다운샘플링(축소)을 수행하는 함수나 레이어
norm_layer: 정규화 레이어의 유형을 지정하는 변수
super: BasicBlock의 부모모듈 nn.Module의 매개변수들을 받아서 초기화한다
'''
class BasicBlock(nn.Module):
    expansion = 1

    # 생성자함수: 인스턴스 생성시 매개변수를 받을수있다
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        
        # 배치정규화설정 
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        # stride가 1이 아니면 레이어들이 다운샘플을 수행한다
        
        # 레이어 초기화
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.actv = nn.ReLU()
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    # 순전파 수행 => 예측 출력
    def forward(self, x):
        # 이미지 x가 입력된다 잔차를 구하기 위해 identity에 저장
        identity = x
        
        # 1차 컨볼루션 수행
        out = self.conv1(x)
        # 배치 정규화: 평균과 분산을 계산하여 출력채널의 데이터 정규화
        out = self.bn1(out)
        #활성화함수적용 = 비선형성 추가
        out = self.actv(out)
        
        #2차 컨볼루션 수행
        out = self.conv2(out)
        out = self.bn2(out)
        
        # 다운 샘플링이 필요할경우 x를 다운샘플링하여 잔차계산을위해 저장해둠
        if self.downsample is not None:
            identity = self.downsample(x)
            
        # 결과와 잔차연결
        out += identity
        # 잔차연결 결과를 ReLU활성화함수에 넣어 최종결과 출력
        out = self.actv(out)

        return out

In [3]:
'''
위에서 만든 Basic블록을 여러번 반복하여 여러개의 레이어를 만드는 유틸리티 함수이다.
resnet 아키텍처에서 사용됨

block: 위에서 정의한 basic블록객체
blocks: 생성할 레이어의 개수
'''
def _make_layer(block, inplanes, planes, blocks, stride=1, dilate=False):
        norm_layer = nn.BatchNorm2d
        downsample = None
        
        # 다운샘플링해서 차원이 맞지 않을때 차원조절한다
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                # 입력 채널수를 planes * block.expansion 로 변환한다
                nn.Conv2d(inplanes, planes * block.expansion, 1, stride, bias=False),
                #배치 정규화 수행
                norm_layer(planes * block.expansion),
            )

        layers = []
        # 블록을 만들어 첫번쨰 레이어를 만든다
        layers.append(block(inplanes, planes, stride, downsample, norm_layer=norm_layer))
        
        # 나머지 blocks-1 개의 레이어를 만든다
        for _ in range(1, blocks):
            layers.append(block(planes, planes, 
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)


'''
이미지 또는 텐서를 원하는 크기로 조절하는 보간(Interpolate)함수
이미지 스케일링 또는 업샘플링을 수행
size: 원하는 목표 크기
mode: 크기를 조절하는 방법

'''
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        # 파이토치의 크기조절연산 함수 사용
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    # 순전파 함수
    def forward(self, x):
        #이미지 x를 받아서 
        '''
        예시 입력 이미지, (배치 크기, 채널 수, 높이, 너비)
        interpolator = Interpolate(size=(256, 256), mode='bilinear')
        output_image = interpolator(torch.randn(1, 3, 128, 128))   # 크기가 (256, 256)으로 조절된 이미지
        align_corners=True : 업샘플링할때 코너의 경계값을 그대로 가져온다
        '''
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=True)
        return x

In [4]:
class GlobalWarper(nn.Module):
    '''
    경계를 잡고 전반적인 문서골곡을 디포메이션한다
    '''
    def __init__(self):
        super(GlobalWarper, self).__init__()
        
        # 레이어 모듈 초기화
        modules = [
            nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        ]

        # encoder
        # 차례대로 사용할 매개변수 지정
        planes = [64, 128, 256, 256, 512, 512]
        strides = [2, 2, 2, 2, 2]
        blocks = [1, 1, 1, 1, 1]
        
        for k in range(len(planes) - 1):
            # k번째 요원소들 가져와서 모듈에 레이어를 만들어서 넣는다
            modules.append(_make_layer(BasicBlock, planes[k], planes[k + 1], blocks[k], strides[k]))
            
        # encoder 시퀀셜 초기화, 모듈스 리스트안의 원소를 분리해서 시퀀셜 인스턴스를 생성
        self.encoder = nn.Sequential(*modules)

        # decoder
        modules = []
        planes = [512, 512, 256, 128, 64]
        strides = [2, 2, 2, 2]
        blocks = [1, 1, 1, 1]
        for k in range(len(planes) - 1):
            # 모듈안에  레이어가 들어간 시퀀셜인스턴스를 넣는다
            modules += [nn.Sequential(nn.Upsample(scale_factor=strides[k], mode='bilinear', align_corners=True), 
                        _make_layer(BasicBlock, planes[k], planes[k + 1], blocks[k], 1))]
            
        # 디코더 초기화 시퀀셜안에 시퀀셜 인스턴스들이 들어가있는 형태다
        self.decoder = nn.Sequential(*modules)
        
        # 디코더 결과 데이터를 변환한다
        self.to_warp = nn.Sequential(nn.Conv2d(64, 2, 1))
        self.to_warp[0].weight.data.fill_(0.0)
        self.to_warp[0].bias.data.fill_(0.0)
        
        '''
        좌표 그리드를 생성하여 워핑 변환에 사용한다
        torch.linspace(-1, 1, 256): -1 부터 1까지를 256등간견으로 나눈 값으로 텐서 생성
        
        '''
        
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, 256), torch.linspace(-1, 1, 256))
        
        # (1, 2, 256, 256)인 텐서로 변환, 좌표이다
        self.coord = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device)
        
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, 64), torch.linspace(-1, 1, 64))
        ### note we mulitply a 0.9 so the network is initialized closer to GT. This is different from localwarper net
        # 원본이미지와 가깝게 초기화하기위해 좌표그리드에 0.9를 곱해서 초기화한다
        self.basegrid = torch.stack((ix * 0.9, iy * 0.9), dim=0).unsqueeze(0).to(device)
        
    # 순전파 함수
    def forward(self, im):
        # print(self.to_warp[0].weight.data)
        # coordconv
        
        # 이미지의 배치크기를(= 이미지 몇개씩 쓸건지) 가져온다
        B = im.size(0)
        
        # 생성된 좌표를 이미지 배치 크기에 맞게 확장해서 c로 초기화한다
        c = self.coord.expand(B, -1, -1, -1).detach()
        # 이미지와 좌표그리드c를 dim=1차원에 따라 결합한다
        # 즉 이미지와좌표정보가 결합된 텐서t가 된다
        t = torch.cat((im, c), dim=1)
        
        # t를 인코더와 엔코더에 넣고 워프변환하기 위해 초기화 한다
        t = self.encoder(t)
        t = self.decoder(t)
        t = self.to_warp(t)
        
        # 최종 t를 basegird에 더하여 최종 좌표그리드 gs를 얻는다 
        gs = t + self.basegrid

        return gs

In [5]:
class LocalWarper(nn.Module):
    '''
    나머지 텍스트의 굴곡등을 텍스트 정렬을 분석하여 정밀하게 포인트를 제어하여 디워핑
    '''
    def __init__(self):
        super().__init__()
        modules = [
            nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        ]
        
        # encoder
        planes = [64, 128, 256, 256, 512, 512]
        strides = [2, 2, 2, 2, 2]
        blocks = [1, 1, 1, 1, 1]
        for k in range(len(planes) - 1):
            modules.append(_make_layer(BasicBlock, planes[k], planes[k + 1], blocks[k], strides[k]))
        self.encoder = nn.Sequential(*modules)

        # decoder
        modules = []
        planes = [512, 512, 256, 128, 64]
        strides = [2, 2, 2, 2]
        # tsizes = [3, 5, 9, 17, 33]
        blocks = [1, 1, 1, 1]
        for k in range(len(planes) - 1):
            modules += [nn.Sequential(nn.Upsample(scale_factor=strides[k], mode='bilinear', align_corners=True), 
                        _make_layer(BasicBlock, planes[k], planes[k + 1], blocks[k], 1))]
        self.decoder = nn.Sequential(*modules)

        self.to_warp = nn.Sequential(nn.Conv2d(64, 2, 1))
        self.to_warp[0].weight.data.fill_(0.0)
        self.to_warp[0].bias.data.fill_(0.0)

        iy, ix = torch.meshgrid(torch.linspace(-1, 1, 256), torch.linspace(-1, 1, 256))
        self.coord = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device)
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, 64), torch.linspace(-1, 1, 64))
        self.basegrid = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device)

        # # box filter => 안씀
        # # 주변 픽셀의 값을 평균화하여 이미지를 부드럽게 만들어줌 => 노이즈감소 + 입력전처리
        # ksize = 5  #커널크기
        # p = int((ksize - 1) / 2)  # 패딩크기
        # # replicate: 이미지의 경계주변의 픽셀값을 패딩에 사용
        # self.pad_replct = partial(F.pad, pad=(p, p, p, p), mode='replicate')
        # #박스필터 가중치 초기화 1 나누기 필터사이즈로 정규화한다
        # bw = torch.ones(1, 1, ksize, ksize, device=device) / ksize / ksize
        # #이미지에 박스 필터를 적용하는 함수로 정의
        # self.box_filter = partial(F.conv2d, weight=bw)

    def forward(self, im):
        c = self.coord.expand(im.size(0), -1, -1, -1).detach()
        t = torch.cat((im, c), dim=1)

        # 인코더 디코더에 넣고 변환 처리
        t = self.encoder(t)
        t = self.decoder(t)
        t = self.to_warp(t)

        # bd condition : 경계조건 설정
        t[..., 1, 0, :] = 0
        t[..., 1, -1, :] = 0
        t[..., 0, :, 0] = 0
        t[..., 0, :, -1] = 0

        gs = t + self.basegrid
        return gs


def gs_to_bd(gs):
    # gs: B 2 H W
    # 좌표그리드의 경계를 추출하고 합친다, 경계를 제어하는데 사용한다
    t = torch.cat([gs[..., 0, :], gs[..., -1, :], gs[..., 1 : -1, 0], gs[..., 1 : -1, -1]], dim=2).permute(0, 2, 1)
    # t: B 2(W + H - 1) 2
    return t

from tps_warp import TpsWarp, PspWarp

class MaskLoss(nn.Module):
    '''
    입력된 이미지와 생성된 좌표그리드사이의 손실계산하는 클래스
    '''
    def __init__(self, gsize):
        super().__init__()
        self.tpswarper = TpsWarp(gsize)
        self.pspwarper = PspWarp()
        # self.imsize = imsize
        self.msk = torch.ones(1, 1, gsize, gsize, device=device)
        self.cn = torch.tensor([[-1, -1], [1, -1], [1, 1], [-1, 1]], dtype=torch.float, device=device).unsqueeze(0)

    def forward(self, gs, y, s):
        # resize gs to s*s
        B, _, s0, _ = gs.size()
        tgs = F.interpolate(gs, s, mode='bilinear', align_corners=True)

        # use only the boundary points
        srcpts = gs_to_bd(tgs)
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, s), torch.linspace(-1, 1, s))
        t = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device).expand_as(tgs)
        dstpts = gs_to_bd(t)

        tgs_f = self.tpswarper(srcpts, dstpts.detach())
        ym = self.msk.expand_as(y)
        yh = F.grid_sample(ym, tgs_f.permute(0, 2, 3, 1), align_corners=True)
        loss_f = F.l1_loss(yh, y)

        # forward/backward consistency loss
        tgs_b = self.tpswarper(dstpts.detach(), srcpts)
        # tgs_b = F.interpolate(tgs, s0, mode='bilinear', align_corners=True)
        yy = F.grid_sample(y, tgs_b.permute(0, 2, 3, 1), align_corners=True)
        loss_b = F.l1_loss(yy, ym)
        
        return loss_f + loss_b, tgs_f

    def _dist(self, x):
        # adjacent point distance
        # B, 2, n
        x = torch.cat([x[..., 0 : 1].detach(), x[..., 1 : -1], x[..., -1 : ].detach()], dim=2)
        d = x[..., 1:] - x[..., :-1]
        return torch.norm(d, dim=1)

ModuleNotFoundError: No module named 'tps_warp'

In [ ]:
class WarperUtil(nn.Module):
    '''
    워핑에 필요한 기능 함수들
    '''
    def __init__(self, imsize):
        super().__init__()
        self.tpswarper = TpsWarp(imsize)
        self.pspwarper = PspWarp()
        self.s = imsize
    
    def global_post_warp(self, gs, s):
        # B, _, s0, _ = gs.size()
        gs = F.interpolate(gs, s, mode='bilinear', align_corners=True)

        # extract info
        m1 = gs[..., 0, :]
        m2 = gs[..., -1, :]
        n1 = gs[..., 0]
        n2 = gs[..., -1]
        # for x
        m1x_interval_ratio = m1[:, 0, 1:] - m1[:, 0, :-1]
        m1x_interval_ratio /= m1x_interval_ratio.sum(dim=1, keepdim=True)
        m2x_interval_ratio = m2[:, 0, 1:] - m2[:, 0, :-1]
        m2x_interval_ratio /= m2x_interval_ratio.sum(dim=1, keepdim=True)
        # interpolate all x ratio
        t = torch.stack([m1x_interval_ratio, m2x_interval_ratio], dim=1).unsqueeze(1)
        mx_interval_ratio = F.interpolate(t, (s, m1x_interval_ratio.size(1)), mode='bilinear', align_corners=True)
        mx_interval = (n2[..., 0 : 1, :] - n1[..., 0 : 1, :]).unsqueeze(3) * mx_interval_ratio
        # cumsum to x
        dx = torch.cumsum(mx_interval, dim=3) + n1[..., 0 : 1, :].unsqueeze(3)
        dx = dx[..., 1 : -1, :-1]
        # for y
        n1y_interval_ratio = n1[:, 1, 1:] - n1[:, 1, :-1]
        n1y_interval_ratio /= n1y_interval_ratio.sum(dim=1, keepdim=True)
        n2y_interval_ratio = n2[:, 1, 1:] - n2[:, 1, :-1]
        n2y_interval_ratio /= n2y_interval_ratio.sum(dim=1, keepdim=True)
        # interpolate all x ratio
        t = torch.stack([n1y_interval_ratio, n2y_interval_ratio], dim=2).unsqueeze(1)
        ny_interval_ratio = F.interpolate(t, (n1y_interval_ratio.size(1), s), mode='bilinear', align_corners=True)
        ny_interval = (m2[..., 1 : 2, :] - m1[..., 1 : 2, :]).unsqueeze(2) * ny_interval_ratio
        # cumsum to y
        dy = torch.cumsum(ny_interval, dim=2) + m1[..., 1 : 2, :].unsqueeze(2)
        dy = dy[..., :-1, 1 : -1]
        ds = torch.cat((dx, dy), dim=1)
        gs[..., 1 : -1, 1 : -1] = ds
        return gs

    def perturb_warp(self, dd):
        B = dd.size(0)
        s = self.s
        # -0.2 to 0.2
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, s), torch.linspace(-1, 1, s))
        t = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device).expand(B, -1, -1, -1)

        tt = t.clone()

        nd = random.randint(0, 4)
        for ii in range(nd):
            # define deformation on bd
            pm = (torch.rand(B, 1) - 0.5) * 0.2
            ps = (torch.rand(B, 1) - 0.5) * 1.95
            pt = ps + pm
            pt = pt.clamp(-0.975, 0.975)
            # put it on one bd
            # [1, 1] or [-1, 1] or [-1, -1] etc
            a1 = (torch.rand(B, 2) > 0.5).float() * 2 -1
            # select one col for every row
            a2 = torch.rand(B, 1) > 0.5
            a2 = torch.cat([a2, a2.bitwise_not()], dim=1)
            a3 = a1.clone()
            a3[a2] = ps.view(-1)
            ps = a3.clone()
            a3[a2] = pt.view(-1)
            pt = a3.clone()
            # 2 N 4
            bds = torch.stack([
                t[0, :, 1 : -1, 0], t[0, :, 1 : -1, -1], t[0, :, 0, 1 : -1], t[0, :, -1, 1 : -1]
            ], dim=2)

            pbd = a2.bitwise_not().float() * a1
            # id of boundary p is on
            pbd = torch.abs(0.5 * pbd[:, 0] + 2.5 * pbd[:, 1] + 0.5).long()
            # ids of other boundaries
            pbd = torch.stack([pbd + 1, pbd + 2, pbd + 3], dim=1) % 4
            # print(pbd)
            pbd = bds[..., pbd].permute(2, 0, 1, 3).reshape(B, 2, -1)            

            srcpts = torch.stack([
                t[..., 0, 0], t[..., 0, -1], t[..., -1, 0], t[..., -1, -1],
                ps.to(device)
            ], dim=2)
            srcpts = torch.cat([pbd, srcpts], dim=2).permute(0, 2, 1)
            dstpts = torch.stack([
                t[..., 0, 0], t[..., 0, -1], t[..., -1, 0], t[..., -1, -1],
                pt.to(device)
            ], dim=2)
            dstpts = torch.cat([pbd, dstpts], dim=2).permute(0, 2, 1)

            tgs = self.tpswarper(srcpts, dstpts)
            tt = F.grid_sample(tt, tgs.permute(0, 2, 3, 1), align_corners=True)

        nd = random.randint(1, 5)
        for ii in range(nd):

            pm = (torch.rand(B, 2) - 0.5) * 0.2
            ps = (torch.rand(B, 2) - 0.5) * 1.95
            pt = ps + pm
            pt = pt.clamp(-0.975, 0.975)

            srcpts = torch.cat([
                t[..., -1, :], t[..., 0, :], t[..., 1 : -1, 0], t[..., 1 : -1, -1],
                ps.unsqueeze(2).to(device)
            ], dim=2).permute(0, 2, 1)
            dstpts = torch.cat([
                t[..., -1, :], t[..., 0, :], t[..., 1 : -1, 0], t[..., 1 : -1, -1],
                pt.unsqueeze(2).to(device)
            ], dim=2).permute(0, 2, 1)
            tgs = self.tpswarper(srcpts, dstpts)
            tt = F.grid_sample(tt, tgs.permute(0, 2, 3, 1), align_corners=True)
        tgs = tt

        # sample tgs to gen invtgs
        num_sample = 512
        # n = (H-2)*(W-2)
        n = s * s
        idx = torch.randperm(n)
        idx = idx[:num_sample]
        srcpts = tgs.reshape(-1, 2, n)[..., idx].permute(0, 2, 1)
        dstpts = t.reshape(-1, 2, n)[..., idx].permute(0, 2, 1)
        invtgs = self.tpswarper(srcpts, dstpts)
        return tgs, invtgs

    def equal_spacing_interpolate(self, gs, s):
        def equal_bd(x, s):
            # x is B 2 n
            v0 = x[..., :-1] # B 2 n-1
            v = x[..., 1:] - x[..., :-1]
            vn = v.norm(dim=1, keepdim=True)
            v = v / vn
            c = vn.sum(dim=2, keepdim=True) #B 1 1
            a = vn / c
            b = torch.cumsum(a, dim=2)
            b = torch.cat((torch.zeros(B, 1, 1, device=device), b[..., :-1]), dim=2)
            
            t = torch.linspace(1e-5, 1 - 1e-5, s).view(1, s, 1).to(device)
            t = t - b # B s n-1
            
            tt = torch.cat((t, -torch.ones(B, s, 1, device=device)), dim=2) # B s n
            tt = tt[..., 1:] * tt[..., :-1] # B s n-1
            tt = (tt < 0).float()
            d = torch.matmul(v0, tt.permute(0, 2, 1)) + torch.matmul(v, (tt * t).permute(0, 2, 1)) # B 2 s
            return d

        gs = F.interpolate(gs, s, mode='bilinear', align_corners=True)
        B = gs.size(0)
        dst_cn = torch.tensor([[-1, -1], [1, -1], [1, 1], [-1, 1]], dtype=torch.float, device=device).expand(B, -1, -1)
        src_cn = torch.stack([gs[..., 0, 0], gs[..., 0, -1], gs[..., -1, -1], gs[..., -1, 0]], dim=2).permute(0, 2, 1)
        M = self.pspwarper.pspmat(src_cn, dst_cn).detach()
        invM = self.pspwarper.pspmat(dst_cn, src_cn).detach()
        pgs = self.pspwarper(gs.permute(0, 2, 3, 1).reshape(B, -1, 2), M).reshape(B, s, s, 2).permute(0, 3, 1, 2)
        t = [pgs[..., 0, :], pgs[..., -1, :], pgs[..., :, 0], pgs[..., :, -1]]
        d = []
        for x in t:
            d.append(equal_bd(x, s))
        pgs[..., 0, :] = d[0]
        pgs[..., -1, :] = d[1]
        pgs[..., :, 0] = d[2]
        pgs[..., :, -1] = d[3]
        gs = self.pspwarper(pgs.permute(0, 2, 3, 1).reshape(B, -1, 2), invM).reshape(B, s, s, 2).permute(0, 3, 1, 2)
        gs = self.global_post_warp(gs, s)
        return gs

In [ ]:
class LocalLoss(nn.Module):
    '''
    '''
    def __init__(self):
        super().__init__()

    def identity_loss(self, gs):
        '''
        입력으로 받은 gs(변환된 좌표 그리드)와 원래의 2D 좌표 그리드 간의 L1 손실을 계산한다
        이 손실은 gs가 원래의 2D 좌표 그리드와 얼마나 가까운지를 나타내며
        이를 통해 변환된 좌표 그리드의 정확성을 평가함
        '''
        s = gs.size(2)
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, s), torch.linspace(-1, 1, s))
        t = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device).expand_as(gs)
        loss = F.l1_loss(gs, t.detach())
        return loss

    def direct_loss(self, gs, invtgs):
        '''
        입력으로 받은 gs (변환된 좌표 그리드)와 역방향으로 변환된 invtgs 간의 L1 손실을 계산
        이 손실은 원래의 좌표 그리드에서 변환된 좌표 그리드로 이동한 후,
        다시 역방향으로 변환하여 원래의 좌표 그리드와 얼마나 가까운지를 평가함
        '''
        loss = F.l1_loss(gs, invtgs.detach())
        return loss

    def warp_diff_loss(self, xd, xpd, tgs, invtgs):
        '''
        입력으로 받은 xd (원본 데이터), xpd (변환된 데이터), tgs (변환된 좌표 그리드),
        그리고 invtgs (역방향으로 변환된 좌표 그리드) 간의 손실을 계산
        이 손실은 원본 데이터를 tgs 좌표 그리드에 매핑하고, 
        반대로 invtgs 좌표 그리드를 사용하여 xpd로 매핑한 결과 간의 L1 손실나타냄
        원본 데이터와 변환된 데이터 간의 일관성을 평가함
        '''
        loss_f = F.l1_loss(xd, F.grid_sample(tgs, xpd.permute(0, 2, 3, 1), align_corners=True).detach())
        loss_b = F.l1_loss(xpd, F.grid_sample(invtgs, xd.permute(0, 2, 3, 1), align_corners=True).detach())
        loss = loss_f + loss_b
        return loss


class SupervisedLoss(nn.Module):
    '''
    
    '''
    def __init__(self):
        super().__init__()
        s = 64
        self.tpswarper = TpsWarp(s)

    def fm2bm(self, fm):
        '''
        주어진 fm (특성 맵)을 바운더리 맵(bm)으로 변환하는 역할
        fm은 특성 맵의 형태로, 여기서는 세 가지 채널을 가정합니다.
        첫 번째 및 두 번째 채널은 x 및 y 좌표의 변환 정보를 나타내며, 
        세 번째 채널은 바운더리 포인트를 나타내는 이진 마스크입니다.
        bm은 주어진 fm의 바운더리 포인트를 따라 변환된 좌표 그리드를 생성합니다.
        '''
        # B 3 N N
        # fm in [0, 1]
        B, _, s, _ = fm.size()
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, s), torch.linspace(-1, 1, s))
        t = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device).expand(B, -1, -1, -1)
        srcpts = []
        dstpts = []
        for ii in range(B):
            # mask
            m = fm[ii, 2]
            # z s
            z = torch.nonzero(m, as_tuple=False)
            num_sample = 512
            n = z.size(0)
            idx = torch.randperm(n)
            idx = idx[:num_sample]
            dstpts.append(t[ii, :, z[idx, 0], z[idx, 1]])
            srcpts.append(fm[ii, : 2, z[idx, 0], z[idx, 1]] * 2 - 1)
        srcpts = torch.stack(srcpts, dim=0).permute(0, 2, 1)
        dstpts = torch.stack(dstpts, dim=0).permute(0, 2, 1)

        bm = self.tpswarper(srcpts, dstpts)

        return bm
    
    def gloss(self, x, y):
        '''
        입력으로 받은 x (원본 데이터)와 y (목표 데이터 또는 변환된 데이터) 간의 손실을 계산
        xbd는 원본 데이터 x를 바운더리 좌표로 변환한 결과이고
        y는 목표 데이터 또는 변환된 데이터임
        xbd와 y 사이의 L1 손실을 계산하여 원본 데이터와 목표 데이터 간의 차이를 평가한다
        '''
        xbd = gs_to_bd(x)
        y = F.interpolate(y, 64, mode='bilinear', align_corners=True)
        
        ybd = gs_to_bd(y).detach()
        loss = F.l1_loss(xbd, ybd.detach())
        return loss

    def lloss(self, x, y, dg):
        '''
        네트워크의 출력인 dg를 사용하여 다양한 손실을 계산
        먼저, 원본 데이터 x와 y 간의 L1 손실을 계산한다음
        dg를 사용하여 원본 데이터를 목표 데이터 y로 변환한 후,
        역방향으로 변환된 좌표 그리드를 생성하여 dl에 저장한다
        dl은 원본 데이터 x와 y 간의 변환 일관성을 나타내며, L1 손실로 측정된다
        네트워크가 원본 데이터를 목표 데이터로 변환하는 데 얼마나 잘 수행되는지를 평가한다
        '''
        # sample tgs to gen invtgs
        B, _, s, _ = dg.size()
        iy, ix = torch.meshgrid(torch.linspace(-1, 1, s), torch.linspace(-1, 1, s))
        t = torch.stack((ix, iy), dim=0).unsqueeze(0).to(device).expand(B, -1, -1, -1)
        num_sample = 512
        # n = (H-2)*(W-2)
        n = s * s
        idx = torch.randperm(n)
        idx = idx[:num_sample]
        srcpts = dg.reshape(-1, 2, n)[..., idx].permute(0, 2, 1)
        dstpts = t.reshape(-1, 2, n)[..., idx].permute(0, 2, 1)
        invdg = self.tpswarper(srcpts, dstpts)
        dl = F.grid_sample(invdg, y.permute(0, 2, 3, 1), align_corners=True)
        dl = F.interpolate(dl, 64, mode='bilinear', align_corners=True)
        loss = F.l1_loss(x, dl.detach())
        return loss, dl.detach()